In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import griddata
import seaborn as sns

%matplotlib inline

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

## 🎯 Concepto de Perfiles 2D

Un **perfil 2D** se construye combinando varios SEV realizados a lo largo de una línea:

```
SEV-1      SEV-2      SEV-3      SEV-4
  |          |          |          |
  ↓          ↓          ↓          ↓
X=0 m     X=20 m     X=40 m     X=60 m
```

### Pasos para generar perfil 2D:
1. **Invertir cada SEV** → Obtener modelos de capas
2. **Expandir modelos** → Convertir capas a puntos (x, z, ρ)
3. **Interpolar** → Rellenar espacio entre puntos
4. **Visualizar** → Gráfico de contornos o pseudosección

---

## 📊 Crear Modelos de Ejemplo

Vamos a simular 4 SEV invertidos con diferentes características geológicas:

In [ ]:
# Modelos sintéticos (resultados de inversión)
modelos = [
    {
        'posicion': 0,
        'nombre': 'SEV-1',
        'espesores': [2, 5, 10],
        'resistividades': [50, 120, 200, 80]
    },
    {
        'posicion': 20,
        'nombre': 'SEV-2',
        'espesores': [3, 6, 12],
        'resistividades': [45, 100, 180, 70]
    },
    {
        'posicion': 40,
        'nombre': 'SEV-3',
        'espesores': [2.5, 7, 15],
        'resistividades': [60, 90, 150, 60]
    },
    {
        'posicion': 60,
        'nombre': 'SEV-4',
        'espesores': [4, 8, 18],
        'resistividades': [55, 85, 130, 50]
    }
]

print("✅ Modelos sintéticos creados")
print(f"📊 Total de SEV: {len(modelos)}")
print(f"📏 Extensión del perfil: 0 - 60 m\n")

# Mostrar resumen de modelos
for modelo in modelos:
    profundidad_max = sum(modelo['espesores'])
    print(f"{modelo['nombre']} (X={modelo['posicion']} m):")
    print(f"   Capas: {len(modelo['resistividades'])}")
    print(f"   Profundidad investigada: {profundidad_max:.1f} m")
    print(f"   Rango resistividad: {min(modelo['resistividades'])} - {max(modelo['resistividades'])} Ω·m\n")

## 🔄 Expandir Modelos a Puntos (x, z, ρ)

Convertimos cada modelo de capas en puntos discretos para interpolación:

In [ ]:
def expandir_modelo(posicion_x, espesores, resistividades, puntos_por_capa=3):
    """
    Convierte modelo de capas en puntos (x, z, rho) para interpolación.
    
    Parámetros:
    -----------
    posicion_x : float
        Posición horizontal del SEV (metros)
    espesores : list
        Espesores de cada capa [h1, h2, ..., hn-1]
    resistividades : list
        Resistividades de cada capa [ρ1, ρ2, ..., ρn]
    puntos_por_capa : int
        Número de puntos por capa para discretización
    
    Retorna:
    --------
    x, z, rho : arrays
        Coordenadas y resistividades de los puntos
    """
    x_points = []
    z_points = []
    rho_points = []
    
    profundidad_actual = 0
    
    # Procesar cada capa
    for i, (espesor, rho) in enumerate(zip(espesores + [espesores[-1]], resistividades)):
        # Generar puntos dentro de la capa
        if i < len(espesores):  # Capa con espesor finito
            prof_fin = profundidad_actual + espesor
            profundidades = np.linspace(profundidad_actual, prof_fin, puntos_por_capa)
        else:  # Última capa (semiespacio infinito)
            prof_fin = profundidad_actual + espesores[-1] * 2  # Extender más allá
            profundidades = np.linspace(profundidad_actual, prof_fin, puntos_por_capa)
        
        # Agregar puntos
        for prof in profundidades:
            x_points.append(posicion_x)
            z_points.append(prof)
            rho_points.append(rho)
        
        profundidad_actual = prof_fin
    
    return np.array(x_points), np.array(z_points), np.array(rho_points)


# Expandir todos los modelos
x_todos = []
z_todos = []
rho_todos = []

for modelo in modelos:
    x, z, rho = expandir_modelo(
        modelo['posicion'],
        modelo['espesores'],
        modelo['resistividades'],
        puntos_por_capa=5
    )
    x_todos.extend(x)
    z_todos.extend(z)
    rho_todos.extend(rho)

# Convertir a arrays
x_todos = np.array(x_todos)
z_todos = np.array(z_todos)
rho_todos = np.array(rho_todos)

print(f"✅ Modelos expandidos")
print(f"📊 Total de puntos: {len(x_todos)}")
print(f"📏 Rango X: {x_todos.min():.1f} - {x_todos.max():.1f} m")
print(f"📏 Rango Z: {z_todos.min():.1f} - {z_todos.max():.1f} m")
print(f"⚡ Rango ρ: {rho_todos.min():.1f} - {rho_todos.max():.1f} Ω·m")

## 🗺️ Visualización de Puntos Discretos

In [ ]:
# Graficar puntos antes de interpolar
fig, ax = plt.subplots(figsize=(14, 6))

# Usar normalización lineal para mejor visualización de colores
from matplotlib.colors import Normalize

scatter = ax.scatter(x_todos, z_todos, c=rho_todos, s=50, 
                     cmap='jet', edgecolors='black', linewidth=0.5,
                     norm=Normalize(vmin=rho_todos.min(), vmax=rho_todos.max()))

# Marcar posiciones de SEV
for modelo in modelos:
    ax.axvline(modelo['posicion'], color='white', linestyle='--', 
               linewidth=2, alpha=0.7, label=modelo['nombre'])

ax.invert_yaxis()
ax.set_xlabel('Distancia (m)', fontsize=12, fontweight='bold')
ax.set_ylabel('Profundidad (m)', fontsize=12, fontweight='bold')
ax.set_title('Puntos Discretos de Modelos SEV', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right', fontsize=9)

cbar = plt.colorbar(scatter, ax=ax, label='Resistividad (Ω·m)')
cbar.set_label('Resistividad (Ω·m)', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 🧮 Interpolación: Comparación de Métodos

Vamos a comparar tres métodos de interpolación:

1. **Linear**: Interpolación lineal por triangulación
   - Rápida y estable
   - Transiciones suaves pero no muy precisas

2. **Cubic**: Interpolación cúbica
   - Transiciones muy suaves
   - Puede crear artefactos (valores fuera de rango)

3. **Nearest**: Vecino más cercano
   - Rápida y sin extrapolación
   - Apariencia "pixelada"

In [ ]:
# Crear grilla de interpolación
x_min, x_max = x_todos.min(), x_todos.max()
z_min, z_max = 0, z_todos.max()

# Resolución de la grilla
nx = 200  # Puntos en X
nz = 100  # Puntos en Z

grid_x = np.linspace(x_min, x_max, nx)
grid_z = np.linspace(z_min, z_max, nz)
grid_x, grid_z = np.meshgrid(grid_x, grid_z)

print(f"📐 Grilla de interpolación creada")
print(f"   Dimensiones: {nx} x {nz} = {nx*nz} puntos")
print(f"   Resolución X: {(x_max-x_min)/nx:.2f} m")
print(f"   Resolución Z: {(z_max-z_min)/nz:.2f} m")

# Interpolar con cada método
metodos = ['linear', 'cubic', 'nearest']
grillas_interpoladas = {}

print("\n⏳ Interpolando...")
for metodo in metodos:
    print(f"   - Método {metodo}...", end=' ')
    
    grilla = griddata(
        points=(x_todos, z_todos),
        values=rho_todos,
        xi=(grid_x, grid_z),
        method=metodo
    )
    
    grillas_interpoladas[metodo] = grilla
    print("✅")

print("\n✅ Interpolación completada")

## 📊 Comparación Visual de Métodos

In [ ]:
# Crear subplots para comparar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Obtener rango global de valores para normalización consistente
vmin_global = rho_todos.min()
vmax_global = rho_todos.max()

for ax, metodo in zip(axes, metodos):
    grilla = grillas_interpoladas[metodo]
    
    # Gráfico de contornos con normalización lineal
    im = ax.contourf(grid_x, grid_z, grilla, levels=20, cmap='jet', 
                     vmin=vmin_global, vmax=vmax_global)
    
    # Contornos de línea
    contours = ax.contour(grid_x, grid_z, grilla, levels=10, colors='white', 
                          linewidths=0.5, alpha=0.4)
    
    # Marcar posiciones SEV
    for modelo in modelos:
        ax.axvline(modelo['posicion'], color='white', linestyle='--', 
                   linewidth=1.5, alpha=0.8)
    
    ax.invert_yaxis()
    ax.set_xlabel('Distancia (m)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Profundidad (m)', fontsize=11, fontweight='bold')
    ax.set_title(f'Interpolación {metodo.capitalize()}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.2)

# Barra de color compartida
fig.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.94, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Resistividad (Ω·m)', fontsize=12, fontweight='bold')

plt.suptitle('Comparación de Métodos de Interpolación', 
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("💡 Observaciones:")
print("   - Linear: Buen balance, transiciones naturales")
print("   - Cubic: Muy suave pero puede crear artefactos")
print("   - Nearest: Preserva valores exactos pero pixelado")

## 🎨 Perfil 2D Final con Anotaciones

In [ ]:
# Usar método lineal para perfil final
grilla_final = grillas_interpoladas['linear']

fig, ax = plt.subplots(figsize=(16, 8))

# Gráfico de contornos rellenos con normalización lineal para mejor paleta
im = ax.contourf(grid_x, grid_z, grilla_final, levels=30, cmap='jet', 
                 vmin=rho_todos.min(), vmax=rho_todos.max())

# Contornos de línea etiquetados
contours = ax.contour(grid_x, grid_z, grilla_final, levels=8, colors='white', 
                      linewidths=1, alpha=0.5)
ax.clabel(contours, inline=True, fontsize=9, fmt='%1.0f Ω·m')

# Marcar posiciones de SEV con líneas y etiquetas
for modelo in modelos:
    ax.axvline(modelo['posicion'], color='white', linestyle='--', 
               linewidth=2, alpha=0.9)
    ax.text(modelo['posicion'], -2, modelo['nombre'], 
            ha='center', va='bottom', color='white', 
            fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))

# Configuración de ejes
ax.invert_yaxis()
ax.set_xlabel('Distancia (m)', fontsize=13, fontweight='bold')
ax.set_ylabel('Profundidad (m)', fontsize=13, fontweight='bold')
ax.set_title('Perfil 2D de Resistividad Eléctrica', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.2, linestyle=':')

# Barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, aspect=30)
cbar.set_label('Resistividad (Ω·m)', fontsize=12, fontweight='bold', rotation=270, labelpad=20)

# Leyenda de interpretación
interpretation_text = """
Interpretación preliminar:
• Azul oscuro (< 50 Ω·m): Arcillas saturadas
• Azul claro (50-100 Ω·m): Arcillas/Limos
• Verde-Amarillo (100-200 Ω·m): Arenas/Gravas
• Rojo (> 200 Ω·m): Material resistivo
"""
ax.text(0.02, 0.98, interpretation_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print("✅ Perfil 2D generado exitosamente")

## 💾 Exportar Perfil para Informes

In [ ]:
# Guardar gráfico en alta resolución
fig.savefig('perfil_2d_resistividad.png', dpi=300, bbox_inches='tight')
print("✅ Gráfico guardado: perfil_2d_resistividad.png (300 dpi)")

# Exportar datos de grilla interpolada
df_grilla = pd.DataFrame({
    'X': grid_x.flatten(),
    'Z': grid_z.flatten(),
    'Resistividad': grilla_final.flatten()
})

df_grilla.to_csv('perfil_2d_datos.csv', index=False)
print("✅ Datos guardados: perfil_2d_datos.csv")

# Crear tabla resumen de modelos
resumen_modelos = []
for modelo in modelos:
    resumen_modelos.append({
        'SEV': modelo['nombre'],
        'Posición (m)': modelo['posicion'],
        'N° Capas': len(modelo['resistividades']),
        'Prof. investigada (m)': sum(modelo['espesores']),
        'ρ mínima (Ω·m)': min(modelo['resistividades']),
        'ρ máxima (Ω·m)': max(modelo['resistividades'])
    })

df_resumen = pd.DataFrame(resumen_modelos)
df_resumen.to_excel('resumen_modelos_sev.xlsx', index=False)
print("✅ Resumen guardado: resumen_modelos_sev.xlsx")

print("\n📊 Resumen de exportación:")
display(df_resumen)

## 🔬 Análisis Estadístico del Perfil

In [ ]:
# Estadísticas de resistividad por profundidad
profundidades_analisis = [0, 5, 10, 20, 30]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Resistividad vs profundidad promedio
resistividades_promedio = []
for prof in profundidades_analisis:
    idx_prof = np.argmin(np.abs(grid_z[:, 0] - prof))
    rho_prof = grilla_final[idx_prof, :]
    resistividades_promedio.append(np.nanmean(rho_prof))

ax1.plot(resistividades_promedio, profundidades_analisis, 'o-', 
         linewidth=2, markersize=8)
ax1.invert_yaxis()
ax1.set_xlabel('Resistividad promedio (Ω·m)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Profundidad (m)', fontsize=11, fontweight='bold')
ax1.set_title('Resistividad Media vs Profundidad', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Subplot 2: Histograma de resistividades
rho_validos = grilla_final[~np.isnan(grilla_final)]
ax2.hist(rho_validos, bins=30, edgecolor='black', alpha=0.7)
ax2.axvline(np.median(rho_validos), color='red', linestyle='--', 
            linewidth=2, label=f'Mediana: {np.median(rho_validos):.1f} Ω·m')
ax2.set_xlabel('Resistividad (Ω·m)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
ax2.set_title('Distribución de Resistividades', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Estadísticas generales
print("📊 Estadísticas del perfil:")
print(f"   Resistividad mínima: {np.nanmin(grilla_final):.1f} Ω·m")
print(f"   Resistividad máxima: {np.nanmax(grilla_final):.1f} Ω·m")
print(f"   Resistividad media: {np.nanmean(grilla_final):.1f} Ω·m")
print(f"   Resistividad mediana: {np.nanmedian(grilla_final):.1f} Ω·m")
print(f"   Desviación estándar: {np.nanstd(grilla_final):.1f} Ω·m")

## 🎯 Ejercicio Práctico

**Tarea**:
1. Carga tus propios modelos invertidos
2. Asigna posiciones X a cada SEV
3. Genera un perfil 2D con interpolación lineal
4. Exporta el resultado en formato PNG y Excel

```python
# Tu código aquí
mis_modelos = [
    {'posicion': 0, 'nombre': 'Mi-SEV-1', 'espesores': [...], 'resistividades': [...]},
    # Agregar más SEV...
]
```

## 📝 Resumen

En este tutorial aprendiste:

✅ **Perfiles 2D**: Combinar múltiples SEV espacialmente  
✅ **Expansión de modelos**: Convertir capas a puntos discretos  
✅ **Interpolación**: Métodos lineal, cúbica y vecino cercano  
✅ **Visualización**: Contornos, anotaciones, interpretación  
✅ **Exportación**: PNG de alta resolución, CSV, Excel  

### 🚀 Siguientes pasos

Ahora puedes:
- Aplicar estos métodos a tus datos reales
- Combinar con información geológica externa
- Crear informes técnicos profesionales

---

**VESPY** - Vertical Electrical Sounding in Python  
📧 josemariagarciamarquez2.72@gmail.com  
🌐 [Web](https://josemariagarciamarquez.github.io/webjoma/) • ☕ [Patreon](https://www.patreon.com/chemitas)